In [1]:
from sionna.rt import load_scene
import mitsuba as mi

import drjit as dr
import numpy as np
import torch

# scene = load_scene(filename="cubes/cubes.xml")
# scene = load_scene(filename="cubes/hugging_cubes.xml")
ray_per_source = 10000

scene = load_scene(filename="minis3/row2_col5.xml")

mi_scene = scene.mi_scene

In [2]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)
print(shapes)

shape_ptrs = dr.reinterpret_array(mi.UInt32, shapes).numpy()
print(shape_ptrs)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []

# shape_ptr_to_offset = np.zeros(shape=shape_ptrs.shape[0]+1, dtype=int)
shape_ptr_to_offset = {}
offset_counter = 0
for i, s in enumerate(shapes):
    shape_ptr_to_offset[shape_ptrs[i]] = offset_counter
    offset_counter += s.primitive_count()

    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
# print(face_averages)
# print(face_normals)

print(shape_ptr_to_offset)


[Mesh[
  name = "element_11634-wall.ply + element_6028-wall.ply + element_6029-wall.ply + element_6031-wall.ply + element_6032-wall.ply + element_6034-wall.ply + element_6037-wall.ply + element_6038-wall.ply + element_6041-wall.ply + element_6045-wall.ply + element_6046-wall.ply + element_6052-wall.ply + element_6053-wall.ply + element_6054-wall.ply + element_6057-wall.ply + element_6058-wall.ply + element_6064-wall.ply + element_6065-wall.ply + element_6068-wall.ply + element_6072-wall.ply + element_6073-wall.ply + element_6075-wall.ply + element_6076-wall.ply + element_6083-wall.ply + element_6084-wall.ply + element_6085-wall.ply + element_6087-wall.ply + element_6089-wall.ply + element_6094-wall.ply + element_6095-wall.ply + element_6096-wall.ply + element_6151-wall.ply + element_6216-wall.ply",
  bbox = BoundingBox3f[
    min = [47.2146, -10.5235, 0],
    max = [284.054, 255.803, 61.4]
  ],
  vertex_count = 390,
  vertices = [4.57 KiB of vertex data],
  face_count = 390,
  faces = 

In [3]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources

sources = face_averages + (face_normals * 0.0001)

sky_test = mi.Ray3f(sources, mi.Vector3f([0,0,1]))
prelim_int = mi_scene.ray_intersect_preliminary(sky_test)
exterior_source = ~prelim_int.is_valid()
exterior_source


[False, False, True, .. 515 skipped .., True, True, True]

In [4]:

rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
# print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
exterior_source = dr.repeat(exterior_source, ray_per_source)
# print(norms_expanded)
# shouldn't this be positive dot prod???
print(rays.d)
active = exterior_source & (dr.dot(rays.d, norms_expanded) > 0)
# print(active)

[[0, 0, 1],
 [-0.0147479, -0.0135103, 0.9998],
 [0.00247273, 0.0281755, 0.9996],
 .. 5209994 skipped ..,
 [-0.0281711, 0.00252272, -0.9996],
 [0.0158941, 0.012141, -0.9998],
 [-0, -0, -1]]


In [5]:
import numpy as np

valid = np.array([True, True, False, True])
print(valid)
indices = np.nonzero(valid)[0]
print(indices)
add_mask = np.array([False, True, True])
print(add_mask)
set_false = indices[~add_mask]
valid[set_false] = False
print(valid)

[ True  True False  True]
[0 1 3]
[False  True  True]
[False  True False  True]


In [6]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)
# print(interactions)
full_int = interactions.compute_surface_interaction(rays, active=active)
# print(full_int.p)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

valid = full_int.is_valid().numpy()
interaction_normals = full_int.n.numpy().T[valid]

# make sure the dot product is negative
# outgoing rays dirs
# maybe keep this in dr.jit
ray_dir_np = rays.d.numpy().T[valid]
print(ray_dir_np)
ray_dir_dot_interaction_normal_np = np.sum(ray_dir_np * interaction_normals, axis=1)
valid_hits_exterior = ray_dir_dot_interaction_normal_np < 0

indices = np.nonzero(valid)[0]
print(indices)

invalid = indices[~valid_hits_exterior]
valid[invalid] = False

# a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
# a=dr.select(full_int.is_valid(), full_int.prim_index, dr.cuda.UInt(count)).numpy()
# interaction_shapes=dr.select(full_int.is_valid(), full_int.shape, dr.cuda.UInt32(36000)).numpy()

indices = np.nonzero(valid)[0]
interaction_shapes = full_int.shape.numpy()[valid]
prim_indices = full_int.prim_index.numpy()[valid]
# interaction_shapes[0].id()
# print(interaction_normals)

[[-2.44426072e-01  1.21734135e-01  9.61996198e-01]
 [-2.37917975e-01  1.61314085e-01  9.57795799e-01]
 [-2.24451870e-01  2.00692013e-01  9.53595340e-01]
 ...
 [-9.71243262e-01  2.38086820e-01  1.10012293e-03]
 [-4.01996404e-01  9.15641129e-01  5.00023365e-04]
 [-9.47280169e-01 -3.20406526e-01  1.00016594e-04]]
[  20190   20211   20232 ... 5204994 5204997 5204999]


In [7]:
# print("shapes:", mi_scene.shapes())
# print(full_int.shape_index)
# # wtf=dr.reinterpret_array(mi.UInt32, full_int.shape)
# # wtf2 = dr.reinterpret_array(mi.ShapePtr, wtf)
# # # print(wtf)
# # for w in wtf2:
# #     print(w.id())
# print(wtf2[3598].has_mesh_attributes())
# np.unique(wtf)

In [8]:
# full_valid = full_int.is_valid().numpy()
# print(full_valid.shape)
# hit_points = full_int.p.numpy().T[full_valid]

# origins = rays.o.numpy().T[full_valid]
# print(origins.shape)

# scene.preview()

# # This cell is lowkey pretty illegal but here's to being easier to implement
# ## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)


# colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
# colors = np.tile(colors, (origins.shape[0], 1))
# # print(colors)
# # print(colors.dtype)
# width = np.array(1.0)
# scene._preview_widget._plot_lines(origins, hit_points, colors, width)

In [9]:
# dir(full_int.prim_index)
# collision_shape_i = full_int.prim_index.numpy()[full_valid]

# np.unique(collision_shape_i)

In [10]:
scene.preview()
None

Next cell could be heavily optimized

In [11]:
sets = [set() for _ in range(count)]

for i, shape_i, prim_i in zip(indices, interaction_shapes, prim_indices):
    # print(i, shape_i, prim_i)
    # shape_i -= 1
    # shape_id = mi_scene.shapes()[shape_i].id()
    offset = shape_ptr_to_offset[shape_i]
    sets[i // ray_per_source].add(offset + prim_i)

# for i in range(len(a)):
#     vis = a[i]
#     if vis < count:
#         cur_shape = interaction_shapes[i] - 1
#         cur_shape = mi_scene.shapes()[cur_shape].id()
#         offset = shape_offset[cur_shape]
#         # offset = mi_scene.shapes[collision_shape_i].id()
#         sets[i // ray_per_source].add(vis + offset)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

[237.79202     0.4362367   8.866667 ] set()
[238.94635    9.256265  17.733334] set()
[246.62234   17.767393   8.866667] {128, 100, 101, 358, 359, 370, 371, 372, 373, 343, 92, 93, 95}
[247.01886   17.715391  17.733334] {128, 100, 101, 358, 359, 370, 371, 372, 373, 343, 92, 93, 95}
[249.7709     9.11741    8.866667] {6, 7, 12, 13, 22, 23}
[249.9173    10.245441  17.733334] {6, 7, 12, 13, 22, 23}
[247.3522     8.286579   8.866667] {4, 5, 12, 13, 22, 23}
[248.48834    8.137979  17.733334] {4, 5, 12, 13, 22, 23}
[249.76859   12.427345   8.866667] {352, 353, 129, 128, 100, 101, 358, 359, 354, 14, 15, 343, 92, 93, 94, 95}
[252.75519   12.033587  17.733334] {352, 353, 129, 128, 100, 101, 358, 359, 357, 14, 15, 92, 343, 349, 93, 94, 95}
[246.18082   17.463194   8.866667] {100, 101, 358, 359, 370, 371, 20, 373, 372, 21, 95}
[246.20331   17.641294  17.733334] {100, 101, 358, 359, 370, 371, 372, 373, 21, 20, 94, 95}
[248.92755   11.522052   8.866667] {4, 5, 6, 7, 22, 23}
[247.79141   11.670631  17

In [ ]:
face_averages


In [12]:
sizes = [len(s) for s in sets]
face_averages_np = face_averages.numpy().T
starts = np.repeat(face_averages_np, repeats=sizes, axis=0)
print(starts.shape)

ends_indices = []
for s in sets:
    for shape in s:
        ends_indices.append(shape)

ends_indices = np.array(ends_indices)
print(ends_indices)
ends = face_averages_np[ends_indices]
ends.shape

print(ends)

(6432, 3)
[128 100 101 ...  80  81 275]
[[225.98184   222.0491      6.2333336]
 [246.80328   219.35959    10.866667 ]
 [237.68338   220.53716    21.733334 ]
 ...
 [146.15746    85.847046   19.333334 ]
 [128.38976    88.26178    38.666668 ]
 [155.79886    19.93689    24.600002 ]]


In [13]:
scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([0.0, 1.0, 0.0, 1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)

In [3]:
# Given a list of locations (x, 3), find visibility edges

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
from sionna.rt import load_scene
import mitsuba as mi

import drjit as dr
import numpy as np
import torch

def antenna_edges(locations, mi_scene, shape_ptr_to_offsets):
    ray_per_source = 10000
    sources = mi.Point3f(locations.T)
    rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources)
    prelim_int = mi_scene.ray_intersect(rays)
    valid = prelim_int.is_valid().numpy()
    prim_indices = prelim_int.prim_index.numpy()[valid]
    shape_i = prelim_int.shape.numpy()[valid]

    #reshape valid into (num tx, ray_per_s)
    valid_2d = valid.reshape((locations.shape[0], -1))
    #sum reduce rows
    sum = np.sum(valid_2d, axis=1)
    # print(sum)
    #cululative add?
    cum_index = np.cumsum(sum)
    # print(cum_index)
    #[5, 0, 3]
    #[5, 5, 8]

    shape_offsets = shape_i
    for key in shape_ptr_to_offset.keys():
        shape_offsets[shape_offsets == key] = shape_ptr_to_offset[key]

    print(np.unique(shape_offsets))

    offsets_plus_prim_i = shape_offsets + prim_indices

    prev = 0
    adjacency_list = []    
    for ci in range(len(cum_index)):
        current = cum_index[ci]
        source_offset_plus_prim_indices = offsets_plus_prim_i[prev:current]
        adjacency_row = np.unique(source_offset_plus_prim_indices)
        adjacency_list.append(adjacency_row)
        prev = current

    #need array of tx specific offsets + 
    #need unique offset + shape_index for each tx
    return adjacency_list
 

pos = np.genfromtxt(fname="output_64maps_10txs_5000rx/row0_col0/tx_locations")
# print(pos)
aj_list = antenna_edges(pos, mi_scene, shape_ptr_to_offset)
print(aj_list)

    

[  0 390]
[array([  3,   8,   9,  18,  19,  20,  21,  44,  45,  47,  52,  53,  60,
        61,  70,  71,  76,  77,  82,  83,  86,  92,  93,  94,  95, 100,
       101, 128, 129, 162, 166, 175, 194, 195, 246, 247, 254, 255, 274,
       275, 276, 277, 280, 281, 291, 296, 311, 317, 328, 329, 330, 331,
       352, 353, 355, 358, 359, 360, 361, 367, 368, 369, 374, 375, 376,
       377, 383, 387, 421, 422, 454], dtype=uint32), array([  8,   9,  14,  15,  18,  19,  20,  21,  44,  45,  52,  53,  60,
        61,  70,  71,  82,  83,  92,  93,  94,  95, 100, 101, 128, 167,
       172, 173, 174, 175, 179, 189, 194, 195, 203, 205, 246, 247, 252,
       254, 255, 274, 275, 276, 277, 278, 279, 280, 281, 291, 296, 300,
       301, 328, 329, 352, 353, 354, 355, 358, 359, 360, 361, 367, 369,
       371, 374, 375, 376, 377, 383, 390, 401, 409, 421, 424, 446, 474,
       476, 479, 482, 509, 511, 514, 516, 518], dtype=uint32), array([  2,   8,   9,  15,  18,  19,  20,  21,  34,  44,  45,  47,  52,
        5

In [4]:
face_averages_np = face_averages.numpy().T

starts = np.repeat(pos[0].reshape((-1,3)), aj_list[0].shape, axis=0)
print(starts)

ends = face_averages_np[aj_list[0]]
print(ends)

[[333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.        ]
 [333.33334351 187.5         30.  

In [5]:
scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([0.0, 1.0, 0.0, 1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)